In [21]:
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

In [22]:
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
BATCH_SIZE = 32
NUM_CLASSES = 21

In [23]:
train_dir = 'train dataset'
val_dir = 'validation dataset'
test_dir = 'test dataset'

In [24]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
                                                    batch_size=BATCH_SIZE,
                                                    class_mode='categorical')

val_generator = val_datagen.flow_from_directory(val_dir,
                                                target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
                                                batch_size=BATCH_SIZE,
                                                class_mode='categorical')

Found 7350 images belonging to 21 classes.
Found 2100 images belonging to 21 classes.


In [25]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3))

model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(NUM_CLASSES, activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [26]:
model.fit(train_generator,
          steps_per_epoch=train_generator.n // train_generator.batch_size,
          epochs=10,
          validation_data=val_generator,
          validation_steps=val_generator.n // val_generator.batch_size)

Epoch 1/10
229/229 [==============================] - 4518s 20s/step - loss: 1.3898 - accuracy: 0.5463 - val_loss: 0.6614 - val_accuracy: 0.7841
Epoch 2/10
229/229 [==============================] - 4818s 21s/step - loss: 0.5227 - accuracy: 0.8293 - val_loss: 0.3990 - val_accuracy: 0.8721
Epoch 3/10
229/229 [==============================] - 3599s 16s/step - loss: 0.3491 - accuracy: 0.8882 - val_loss: 0.2629 - val_accuracy: 0.9106
Epoch 4/10
229/229 [==============================] - 3490s 15s/step - loss: 0.2663 - accuracy: 0.9143 - val_loss: 0.2422 - val_accuracy: 0.9298
Epoch 5/10
229/229 [==============================] - 3568s 16s/step - loss: 0.1984 - accuracy: 0.9378 - val_loss: 0.4928 - val_accuracy: 0.8918
Epoch 6/10
229/229 [==============================] - 3549s 16s/step - loss: 0.1963 - accuracy: 0.9371 - val_loss: 0.1632 - val_accuracy: 0.9447
Epoch 7/10
229/229 [==============================] - 3537s 15s/step - loss: 0.1544 - accuracy: 0.9509 - val_loss: 0.1126 - val_ac

In [30]:
model.save('VGG16.h5')

In [31]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
                                                  batch_size=BATCH_SIZE,
                                                  class_mode='categorical',
                                                  shuffle=False)

predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

Found 1050 images belonging to 21 classes.


In [33]:
print('Confusion Matrix')
print(confusion_matrix(true_classes, predicted_classes))

print('\nClassification Report')
print(classification_report(true_classes, predicted_classes, target_names=class_labels))

Confusion Matrix
[[49  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 50  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0 48  1  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 49  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0]
 [ 0  0  0  0 48  0  0  0  0  0  0  0  0  0  2  0  0  0  0  0  0]
 [ 0  0  0  0  0 50  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  2  0 42  0  0  0  0  1  3  2  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 49  0  0  0  0  0  0  0  0  1  0  0  0  0]
 [ 0  0  0  0  0  0  0  0 50  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  1  1  0  0  0  0  0 47  0  0  0  0  0  0  1  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 49  0  0  0  0  0  0  0  0  1  0]
 [ 0  0  0  0  0  0  0  0  0  0  0 50  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  1  0  0  0  0  0  0  0 49  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  1  0  0  0  0  0  0  0  1 46  0  0  0  0  0  2  0]
 [ 0  0  0  0  0  0  0  0  2  0  0  0  0  0 48  0  0  0  0 

In [37]:
f1 = f1_score(true_classes, predicted_classes, average='macro')
precision = precision_score(true_classes, predicted_classes, average='macro')
recall = recall_score(true_classes, predicted_classes, average='macro')

print('F1 Score:', f1)
print('Precision:', precision)
print('Recall:', recall)

F1 Score: 0.973207234628312
Precision: 0.9743693759146543
Recall: 0.9733333333333334


In [36]:
sample_image_path = 'test image path'
sample_image = tf.keras.preprocessing.image.load_img(sample_image_path, target_size=(IMAGE_WIDTH, IMAGE_HEIGHT))
sample_image = tf.keras.preprocessing.image.img_to_array(sample_image)
sample_image = np.expand_dims(sample_image, axis=0)
sample_image = sample_image / 255.0

prediction = model.predict(sample_image)
predicted_class_index = np.argmax(prediction)
predicted_class_name = class_labels[predicted_class_index]

print('\nPrediction')
print('Predicted class name:', predicted_class_name)


Prediction
Predicted class name: river
